In [4]:
import shapely 
colors= ['#ffffff', '#fff0f0', '#f0fff0', '#f0f0ff']

p= figure(width=900, height=700)
for i, ward in wards.iterrows():
    polygons = ward['geometry']
    if isinstance(polygons, shapely.geometry.polygon.Polygon): polygons = [polygons]
    
    for polygon in polygons:
        longitudes. latitudes = zip(*list(polygon.exterior.coords))
        p.patch(longitudes, latitudes, fill_color=colors[i%4], line_color='black',line_width=.5)
show(p)

NameError: name 'wards' is not defined

In [3]:
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
import geopandas as gp

def getPolyCoords(row, geom, coord_type):
    if coord_type == 'x':
        return list(row[geom].exterior.coords.xy[0])
    elif coord_type == 'y':
        return list(row[geom].exterior.coords.xy[1])

gdf = gp.GeoDataFrame.from_file('map.shp')
gdf['x'] = gdf.apply(getPolyCoords, geom = 'geometry', coord_type = 'x', axis = 1)
gdf['y'] = gdf.apply(getPolyCoords, geom = 'geometry', coord_type = 'y', axis = 1)
p_df = gdf.drop('geometry', axis = 1).copy()
p = figure(title = "A map from Shapefile", plot_width = 800)
p.patches('x', 'y', source = ColumnDataSource(p_df), line_color = "white", line_width = 0.5)
show(p)

DriverError: map.shp: No such file or directory

In [5]:
from bokeh.plotting import figure, output_file, show, reset_output, save, ColumnDataSource
from bokeh.models.tools import HoverTool
from bokeh.transform import factor_cmap
from bokeh.palettes import Category20b		# color palette
from bokeh.palettes import Greys			# color palette
import pandas as pd
import requests
import numpy as np
import re

#data
url = 'https://www.basketball-reference.com/leagues/NBA_2020_per_game.html#per_game_stats::pts_per_g' # Player Status Points Per Game
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[-1]		#select the first table data from the website

to_drop = ['PTS']
df=df[~df['PTS'].isin(to_drop)]		#delet header rows
df['PTS'] = pd.to_numeric(df['PTS'], errors='coerce')	#change datatype to number 
df_PTS = df.nlargest(20,'PTS')		#extract top twenty twos in terms of Colume TPS's value
df_PTS = df_PTS.iloc[::-1]
print(df_PTS)

#insert column for profile picture website
#df_PTS.insert(30, 'Profile', 'https://d2cwpp38twqe55.cloudfront.net/req/201910291/images/players/adamsst01.jpg')
Player_add = []
translationTable = str.maketrans("čćöášý", "ccoasy")
# edit profile website string
for x in range(len(df_PTS)):
	player_name_ori = re.sub("[^a-z ]", "", df_PTS.iloc[x]['Player'].lower().translate(translationTable))	#make letters lowercase and translare letters from other language to english and delet all non-alphabet letters
	player_name_list=player_name_ori.split(" ")
	if(player_name_ori=='anthony davis' or player_name_ori=='kemba walker' or player_name_ori=='lou williams'):		# exception on anthony davis since his profile picture is ..davisan02.html
		player_name_finial = "https://d2cwpp38twqe55.cloudfront.net/req/201910291/images/players/" + player_name_list[1][:5]+player_name_list[0][:2] +"02.jpg"
		Player_add.insert(x,player_name_finial)
	else:
		player_name_finial = "https://d2cwpp38twqe55.cloudfront.net/req/201910291/images/players/" + player_name_list[1][:5]+player_name_list[0][:2] +"01.jpg"
		Player_add.insert(x,player_name_finial)

	#print(df_PTS['Profile'][0])
#################################################### need change dataframe data 11/15 6pm
#df_PTS.insert(30, 'Profile', Player_add)

df_PTS['Profile']=Player_add		# insert new column for Profile picture address
print(df_PTS['Profile'])

df_PTS['Color'] = Category20b[len(df_PTS)][::-1]		# insert one colum for bar color 
#df_PTS['Color'] = bokeh.palettes.grey(len(df_PTS))		# insert one colum for bar color 
print(df_PTS['Color'])
#Create ColumDataSource from dataframe. instead of using pts = df_PTS['PTS']
source_pts = ColumnDataSource(df_PTS)


output_file('PointsPerGame2019-2020.html')
playerlist = source_pts.data['Player'].tolist()
# Add a plot
p = figure(
	y_range = playerlist,  # pass player list to y_range 
	plot_width = 1000,
	plot_height = 800,
	title = 'NBA Data Visualization',
	x_axis_label = 'Points Per Game',
	y_axis_label = 'Top 20 Players in PTS',
	tools = 'pan, box_select, zoom_in, zoom_out, save, reset',
	match_aspect=True,
	min_border_top = 120,		# leave padding to the figure
	min_border_bottom = 0,
	min_border_left = 100,
	min_border_right = 60
	)

p.hbar (
	y = 'Player',
	right = 'PTS',
	left = 0,
	height = 0.6,
	source = source_pts,
	fill_color = 'Color',
    fill_alpha=0.5
)

# Add interaction
hover = HoverTool()
hover.tooltips = """
	<div>
		<h3>@Player</h3>
		<div><strong>Points Per Game: </strong>@PTS{0.2f}</div>	
		<div><strong>Team: </strong>@Tm</div>
		<div><strong>Position: </strong>@Pos</div>
		<div><strong>Age: </strong>@Age</div>
		<div><img src="@Profile" alt="" width="80" /></div>
	</div>
"""
p.add_tools(hover)

# Show plot
show(p)
	
#df_data = pd.DataFrame(df_PTS.to_numpy())		#change numpy array to dataframe
#df_data.to_excel('pandas_playerdat.xlsx', index=False)		#save dataframe data to excel


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


      Rk                 Player Pos Age   Tm   G  GS    MP    FG   FGA  ...  \
156  139            Joel Embiid   C  25  PHI  31  31  31.1   7.7  16.3  ...   
452  417          Pascal Siakam  PF  25  TOR  34  34  35.1   8.8  19.3  ...   
186  165            Paul George  SF  29  LAC  26  26  30.7   7.6  17.8  ...   
440  405       D'Angelo Russell  SG  23  GSW  29  29  32.2   8.4  19.4  ...   
356  325       Donovan Mitchell  SG  23  UTA  43  43  34.3   9.1  19.8  ...   
301  273            Zach LaVine  SG  24  CHI  47  47  34.1   8.6  19.4  ...   
263  237           LeBron James  PG  35  LAL  43  43  34.8   9.7  19.6  ...   
254  228         Brandon Ingram  PF  22  NOP  41  41  34.6   8.6  18.4  ...   
511  467      Russell Westbrook  PG  31  HOU  38  38  35.6   9.8  21.8  ...   
255  229           Kyrie Irving  PG  27  BRK  16  16  32.8   9.6  20.9  ...   
132  116          Anthony Davis  PF  26  LAL  38  38  34.5   9.1  18.0  ...   
482  440     Karl-Anthony Towns   C  24  MIN  28  28

In [6]:
df_PTS

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Profile,Color
156,139,Joel Embiid,C,25,PHI,31,31,31.1,7.7,16.3,...,9.8,12.3,3.3,0.8,1.4,3.0,3.5,23.4,https://d2cwpp38twqe55.cloudfront.net/req/2019...,#de9ed6
452,417,Pascal Siakam,PF,25,TOR,34,34,35.1,8.8,19.3,...,6.5,7.6,3.5,0.9,1.0,2.6,2.8,23.5,https://d2cwpp38twqe55.cloudfront.net/req/2019...,#ce6dbd
186,165,Paul George,SF,29,LAC,26,26,30.7,7.6,17.8,...,5.5,6.0,3.7,1.5,0.5,3.3,2.5,23.5,https://d2cwpp38twqe55.cloudfront.net/req/2019...,#a55194
440,405,D'Angelo Russell,SG,23,GSW,29,29,32.2,8.4,19.4,...,3.2,3.6,6.3,0.8,0.3,3.1,2.1,23.9,https://d2cwpp38twqe55.cloudfront.net/req/2019...,#7b4173
356,325,Donovan Mitchell,SG,23,UTA,43,43,34.3,9.1,19.8,...,3.5,4.2,4.2,1.1,0.3,2.4,2.3,24.7,https://d2cwpp38twqe55.cloudfront.net/req/2019...,#e7969c
301,273,Zach LaVine,SG,24,CHI,47,47,34.1,8.6,19.4,...,4.1,4.8,3.8,1.4,0.5,3.2,2.2,24.9,https://d2cwpp38twqe55.cloudfront.net/req/2019...,#d6616b
263,237,LeBron James,PG,35,LAL,43,43,34.8,9.7,19.6,...,6.6,7.7,10.8,1.3,0.5,3.8,1.7,25.2,https://d2cwpp38twqe55.cloudfront.net/req/2019...,#ad494a
254,228,Brandon Ingram,PF,22,NOP,41,41,34.6,8.6,18.4,...,5.6,6.5,4.3,0.9,0.8,3.0,3.0,25.2,https://d2cwpp38twqe55.cloudfront.net/req/2019...,#843c39
511,467,Russell Westbrook,PG,31,HOU,38,38,35.6,9.8,21.8,...,6.5,8.1,7.4,1.6,0.3,4.3,3.6,25.8,https://d2cwpp38twqe55.cloudfront.net/req/2019...,#e7cb94
255,229,Kyrie Irving,PG,27,BRK,16,16,32.8,9.6,20.9,...,4.1,5.2,6.7,1.4,0.6,2.3,2.4,26.1,https://d2cwpp38twqe55.cloudfront.net/req/2019...,#e7ba52
